In [24]:
import re
import pandas as pd
pd.set_option('display.max_rows', None)
with open('output-grid-causal-2.txt') as fhand:
    file = fhand.read()
op = re.split(r'\n\n\n',file)
op[0]

'parameter(adv_step, adv_epoch, adv_epoch_full, lr, ro): (2048.0, 10.0, 10.0, 1e-06, 1e-06)\ngender/race combined consistency 0\nspouse consistency 0\nacc_temp: 0.39115533443891654, bal_acc_temp: 0.4883667542293745, race_gap_rms_temp: 0.007396053779178127, race_max_gap_temp: 0.008405974510597547, gender_gap_rms_temp: 0.18381555276619255, gender_max_gap_temp: 0.2108603170665478'

In [25]:
def string_format(x):
    s = re.findall(r'(\d+)\.(\d+)|(\d+)e-(\d+)|(\d)\.(\d+)e-(\d+)|(\d+)',x)
    string_s = []
    for i in s:
        if i[0] != '':
            string_s.append(i[0]+'.'+i[1])
        if i[2] != '':
            string_s.append(i[2] + 'e-'+i[3])
        if i[4] != '':
            string_s.append(i[4]+'.'+i[5]+'e-'+i[6])
        if i[7] !='':
            string_s.append(i[7])
    return(string_s)
ops = [string_format(x) for x in op]
ops = ops[:(len(ops)-1)]

In [26]:
output_dict = dict()
for op in ops:
    for i in range(len(op)):
        op[i] = float(op[i])
    output_dict[tuple(op[:5])] = op[5:]

In [27]:
df_output = pd.DataFrame.from_dict(output_dict, orient='index', columns=['gender/race combined consistency','spouse consistency', 'acc-temp' , 'bal-acc-temp', 'race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.rename_axis('(adv_step, adv_epoch, adv_epoch_full, lr, ro)')
df_output = df_output.sort_values(by=['race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.sort_values(by=['bal-acc-temp'],ascending=False)

Outputs for adult data with sensr algorithm. Sensetive directions (directions for both gender and race) are from causal regression: X_gender_race_train ~ y_gender_race_train

In [28]:
df_output[['acc-temp' , 'bal-acc-temp', 'race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap']]

,acc-temp,bal-acc-temp,race-gap-rms,race-max-gap,gender-gap-rms,gender-max-gap
"(adv_step, adv_epoch, adv_epoch_full, lr, ro)",,,,,,
"(4.0, 10.0, 10.0, 0.01, 0.01)",0.805860,0.822052,0.055112,0.077926,0.154946,0.199518
"(4.0, 10.0, 10.0, 0.01, 0.0001)",0.809287,0.818451,0.046482,0.060431,0.115795,0.160284
"(4.0, 10.0, 10.0, 0.0001, 0.0001)",0.774572,0.774991,0.036266,0.049477,0.037419,0.051858
"(4.0, 10.0, 10.0, 0.0001, 1e-06)",0.762742,0.753903,0.051951,0.069196,0.068207,0.090182
"(32.0, 10.0, 10.0, 0.0001, 0.0001)",0.732338,0.724095,0.032062,0.043643,0.120647,0.145773
"(32.0, 10.0, 10.0, 0.0001, 1e-06)",0.695965,0.721674,0.060388,0.078158,0.129525,0.145102
"(32.0, 10.0, 10.0, 0.01, 0.0001)",0.734992,0.720291,0.046117,0.065142,0.137668,0.161215
"(32.0, 10.0, 10.0, 0.01, 0.01)",0.719845,0.712909,0.042691,0.056294,0.151251,0.181670
"(4.0, 10.0, 10.0, 1e-06, 1e-06)",0.690105,0.625661,0.039708,0.047721,0.129643,0.182719
